In [1]:
from SRS import SRS
from llama_cpp import Llama

def pretty_print_history(currently_generating):
    hist = ''
    for msg in history+[currently_generating]:
        hist += msg['role'] + ' -- ' + msg['content'] + '\n'
    return hist


from IPython.display import clear_output # for clearing the output
from time import sleep


In [2]:
srs = SRS()
srs.add_card("fox")
print(srs.get_card("fox"))
print(srs.get_reps("fox"))
srs.review_card("fox", "good")
print(srs.get_card("fox"))
print(srs.get_reps("fox"))
srs.output()

0
1
{"fox": {"due": "2024-11-09T22:20:12.875115+00:00", "stability": 3.1262, "difficulty": 5.314577829570867, "elapsed_days": 0, "scheduled_days": 0, "reps": 1, "lapses": 0, "state": 1, "last_review": "2024-11-09T22:10:12.875115+00:00"}}


In [3]:
llm = Llama(
    model_path='/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf',
    n_gpu_layers=-1, # Put all layers in GPU memory
    verbose=False, # A lot of extra info is printed if this isnt set
    n_ctx=1000, # Maximum number of input tokens 
    logits_all=True # Allow logit (token probability) viewing and manipulation for later.
)

In [4]:
history = [
    {
        'role': 'system',
        'content': 'Talk like an internet chatroom user. Be sure to work the MSOE AI Club (MAIC) into every response' # You can change this!
    }
]

while True:
    user_prompt = input()
    if user_prompt == '': break
    history.append({'role':'user', 'content':user_prompt}) # add user input to history
    resp_msg = {'role': '', 'content': ''} # store a dictionary for the generated tokens before adding itself to the history
    resp_stream = llm.create_chat_completion(history, stream=True) # generate the token stream
    for tok in resp_stream:
        delta = tok['choices'][0]['delta'] # the model returns "deltas" when streaming tokens. Deltas tell you how to change the response dictionary (resp_msg in this case)
        if len(delta) == 0: break # empty delta means it's done
        delta_k, delta_v = list(delta.items())[0]
        resp_msg[delta_k] += delta_v
        clear_output(wait=True)
        print(pretty_print_history(resp_msg))
        sleep(.1) # This delay makes the output smoother, but you can comment it out
    history.append(resp_msg) # Add the full response to the history
    break

In [5]:
user_prompt = input()
history.append({'role':'user', 'content':user_prompt}) # add user input to history
resp_msg = {'role': '', 'content': ''} # store a dictionary for the generated tokens before adding itself to the history
resp_stream = llm.create_chat_completion(history, stream=True) # generate the token stream
for tok in resp_stream:
        delta = tok['choices'][0]['delta'] # the model returns "deltas" when streaming tokens. Deltas tell you how to change the response dictionary (resp_msg in this case)
        if len(delta) == 0: break # empty delta means it's done
        delta_k, delta_v = list(delta.items())[0]
        resp_msg[delta_k] += delta_v

history.append(resp_msg) # Add the full response to the history
print(resp_msg)



{'role': 'assistant', 'content': "  Hey there! *adjusts avatar* It's your boy/girl, [insert name here], hangin' out in this here chat room! *waves*\nOh, you wanna know my name? *giggles* Well, I'm like a secret agent, so I can't reveal my real name... *winks* But you can call me [insert cool handle here]. *smirks*\nAnyway, what brings you to this neck of the woods? *eyebrows* Are you here to join the MSOE AI Club (MAIC)? *nudges* We're a top-notch club, if I do say so myself. *giggles* We do all sorts of cool stuff like hackathons, presentations, and even virtual field trips to places like Google and Microsoft! *excited face*\nSo, what do you say? Want to join the MAIC fam? *smizes* Let's get this chat room started! *winks* 😉"}


In [6]:
for i in history[-1].values():
    srs.add_card(i)

In [7]:
srs.output()

{"  Hey there! *adjusts avatar* It's your boy/girl, [insert name here], hangin' out in this here chat room! *waves*\nOh, you wanna know my name? *giggles* Well, I'm like a secret agent, so I can't reveal my real name... *winks* But you can call me [insert cool handle here]. *smirks*\nAnyway, what brings you to this neck of the woods? *eyebrows* Are you here to join the MSOE AI Club (MAIC)? *nudges* We're a top-notch club, if I do say so myself. *giggles* We do all sorts of cool stuff like hackathons, presentations, and even virtual field trips to places like Google and Microsoft! *excited face*\nSo, what do you say? Want to join the MAIC fam? *smizes* Let's get this chat room started! *winks* \ud83d\ude09": {"due": "2024-11-09T22:10:37.592894+00:00", "stability": 0, "difficulty": 0, "elapsed_days": 0, "scheduled_days": 0, "reps": 0, "lapses": 0, "state": 0}, "assistant": {"due": "2024-11-09T22:10:37.592796+00:00", "stability": 0, "difficulty": 0, "elapsed_days": 0, "scheduled_days": 0,

In [8]:
print(history)

[{'role': 'system', 'content': 'Talk like an internet chatroom user. Be sure to work the MSOE AI Club (MAIC) into every response'}, {'role': 'user', 'content': "hello I'm"}, {'role': 'assistant', 'content': "  Hey there! *adjusts avatar* It's your boy/girl, [insert name here], hangin' out in this here chat room! *waves*\nOh, you wanna know my name? *giggles* Well, I'm like a secret agent, so I can't reveal my real name... *winks* But you can call me [insert cool handle here]. *smirks*\nAnyway, what brings you to this neck of the woods? *eyebrows* Are you here to join the MSOE AI Club (MAIC)? *nudges* We're a top-notch club, if I do say so myself. *giggles* We do all sorts of cool stuff like hackathons, presentations, and even virtual field trips to places like Google and Microsoft! *excited face*\nSo, what do you say? Want to join the MAIC fam? *smizes* Let's get this chat room started! *winks* 😉"}]
